In [ ]:
import requests
import pandas as pd
import json

# Example usage
api_key = 'bb4e15af4d57941a734fd8e8156aa737'  # Replace with your actual API key
#For nashik
lat = '19.997454'  # Latitude for the desired location
lon = '73.789803'  # Longitude for the desired location

url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}' # <-- This is app id also known as api key, which you have to generate your own, and paste it here

#vurl ='https://api.openweathermap.org/data/2.5/weather?lat=44.34&lon=10.99&appid=307d11bca480dd730d99187c926&city_name=London'
response = requests.get(url)
data = response.json()
pretty_json = json.dumps(data,indent=4)
print(pretty_json)

In [ ]:
weather_list = []

# Extract relevant information for each forecast entry
for entry in data['list']:
    temperature = entry['main']['temp']
    humidity = entry['main']['humidity']
    wind_speed = entry['wind']['speed']
    weather_description = entry['weather'][0]['description']
    date_time = entry['dt_txt']  # This is the date and time in 'YYYY-MM-DD HH:MM:SS' format
    
    # Split date_time into date and time
    date, time = date_time.split(' ')
    
    # Append the extracted information as a dictionary
    weather_list.append({
        'Temperature': temperature,
        'Humidity': humidity,
        'Wind Speed': wind_speed,
        'Weather Description': weather_description,
        'Date Time': date_time,
        'Date': date,
        'Time': time
    })

# Create a DataFrame from the weather_list
weather_df = pd.DataFrame(weather_list)

# Display the DataFrame
weather_df.head()

In [ ]:
# Fill missing values or drop rows if needed (example: fill with the mean)
# Check for missing values
print(weather_df.isnull().sum())

# Fill missing values or drop rows if needed (example: fill with the mean)
# Using .loc to select multiple columns correctly
weather_df[['Temperature', 'Humidity', 'Wind Speed']] = weather_df[['Temperature', 'Humidity', 'Wind Speed']].fillna(weather_df[['Temperature', 'Humidity', 'Wind Speed']].mean())

# Ensure data types are correct (optional)
weather_df['Temperature'] = weather_df['Temperature'].astype(float)
weather_df['Humidity'] = weather_df['Humidity'].astype(float)
weather_df['Wind Speed'] = weather_df['Wind Speed'].astype(float)

weather_df.head()

In [ ]:
weather_df.info()

In [ ]:
# Calculate average temperature
average_temperature = weather_df['Temperature'].mean()
print(f'Average Temperature: {average_temperature:.2f}°C')

# Maximum and minimum temperature
max_temperature = weather_df['Temperature'].max()
min_temperature = weather_df['Temperature'].min()
print(f'Max Temperature: {max_temperature:.2f}°C, Min Temperature: {min_temperature:.2f}°C')
weather_df.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Line chart for temperature changes over time
plt.figure(figsize=(12, 6))
plt.plot(weather_df['Date Time'], weather_df['Temperature'], marker='o', linestyle='-')
plt.title('Temperature Changes Over Time')
plt.xlabel('Date Time')
plt.ylabel('Temperature (°C)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Bar plot for humidity
plt.figure(figsize=(12, 6))
plt.bar(weather_df['Date Time'], weather_df['Humidity'], color='skyblue')
plt.title('Humidity Levels Over Time')
plt.xlabel('Date Time')
plt.ylabel('Humidity (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Bar plot for windspeed
plt.figure(figsize=(12, 6))
plt.bar(weather_df['Date Time'], weather_df['Wind Speed'], color='skyblue')
plt.title('Wind Speed Over Time')
plt.xlabel('Date Time')
plt.ylabel('Wind Speed (km/hr)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Convert 'Date Time' to datetime format
weather_df['Date Time'] = pd.to_datetime(weather_df['Date Time'])

# Aggregate daily data
daily_summary = weather_df.resample('D', on='Date Time').agg({
    'Temperature': ['mean', 'max', 'min'],
    'Humidity': 'mean',
    'Wind Speed': 'mean'
}).reset_index()

print(daily_summary)

In [ ]:
# Correlation matrix
correlation = weather_df[['Temperature', 'Humidity', 'Wind Speed']].corr()

# Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation between Weather Attributes')
plt.show()